# Notebook 7 - Crear y consultar una Base de Datos no relacional orientada a Documentos (Mongo)

## Ejercicio 1: Primeros pasos desde Mongo CLI

En un terminal, verificar si el SGBD Mongo ya está iniciado:

<code>ps -aux | grep mongod</code>

Si no está iniciado, iniciarlo con el comando: <code>mongod</code>

Luego, abrir otra pestaña de terminal (CTRL + Shift + T) e iniciar el cliente CLI de mongo: <code>mongo</code>.

1) Creamos una nueva base de datos llamado <b>nb7-peliculas</b>

<code>use nb7-peliculas</code>

Deberian ver como respuesta del sistema la respuesta: "switched to db nb7-peliculas"

Mongo CLI no utiliza SQL para realizar operaciones CRUD (Create/Read/Update/Delete), utiliza su propia API parecida al lenguaje Javascript.

La variable <code>db</code> representa siempre la base de datos que estamos utilizando (en este caso "nb7-peliculas").

2) Creamos una colección <b>actors</b>

<code>db.actors.insert({firstname:"Will", lastname:"Smith"})</code>

Como pueden verlo, para crear la colección no tuvimos que definir la estructura de la colección, simplemente hemos insertado un documento.

Verificamos que hemos realmente creado la coleccion con:

<code>db.actors.find()</code>

3) Insertamos varios documentos en la colección en un solo comando:

<code>db.actors.insertMany([{firstname:"Scarlett", lastname:"Johanson"},{firstname:"Matt", lastname:"Damon"},{firstname:"Nathan", lastname:"Smith"}])</code>

4) Contamos cuántos documentos tenemos en la colección:

<code>db.actors.count()</code>

Como en SQL, la API de Mongo permite construir consultas para filtrar, ordenar o agrupar datos. Sin embargo, la síntaxis es distinta (y amenudo menos simple e intuitiva que SQL).

5) Filtramos los actores que se llaman 'Smith'

<code>db.actors.find({lastname:"Smith"})</code>

Mongo tambien permite modificar los datos (ojo: Mongo no tiene mecanismo interno para asegurar la coherencia de los datos, es la responsabilidad del programador de la aplicación).

6) Actualizamos un documento. Por ejemplo, "Nathan Smith" se llama ahora "Nathan Lane"

<code>db.actors.update({firstname:"Nathan", lastname:"Smith"},{$set : {lastname:"Lane"}})</code>

Este último comando va a actualizar el primer documento encontrado que corresponde al filtro definido.

7) Supongamos ahora que queremos agregar un campo "gender" en todos los documentos:

¿Qué hace el comando siguiente? ¿Cuál es el limite?

<code>db.actors.update({},{$set : {gender:"M"}})</code>

Para actualizar todos los documentos, y no simplemente el primero, podemos utilizar el comando:

<code>db.actors.update({},{$set : {gender:"M"}},{multi:true})</code>

8) Actualizar el documento "Scarlett Johanson", para indicar que es una mujer (gender:"F").

9) Para suprimir un atributo, podemos utilizar el operador $unset. Por ejemplo:
    
<code>db.actors.update({},{$unset : {gender:1}},{multi:true})</code>

10) Para suprimir un documento completo, podemos utilizar el comando:

<code>db.actors.remove({lastname:"Smith"})</code>

11) Agregamos un atributo de tipo arreglo en un documento

<code>db.actors.update({firstname:"Matt"},{$set : {tab:[1,"a"]}})</code>

12) Añadimos un valor en el arreglo con el operator $push

<code>db.actors.update({firstname:"Matt"},{$push : {tab:["new_value"]}})</code>

¿Cuál es el problema con el último comando? ¿Cuál es la diferencia con el comando siguiente?

<code>db.actors.update({firstname:"Matt"},{$push : {tab:"new_value"}})</code>

Cómo pueden verlo el SGBD no relacional Mongo es mucho más flexible que MySQL, pero deja mucho más posibilidad de equivocarse.


No duden en consultar la documentación de Mongo para probar otros comandos CRUD: https://docs.mongodb.com/manual/crud/


## Ejercicio 2: Primeras consultas avanzadas con una colección de peliculas

En este segundo ejercicio, trabajaremos con una base de datos existente llamada "movie details", la cual tiene 2295 documentos JSON resumiendo información sobre peliculas.

- Descargar el dataset https://raw.githubusercontent.com/steveren/docs-assets/charts-tutorial/movieDetails.json. ¿Qué contiene este archivo? Un documento JSON? Un arreglo de documentos JSON? Una lista de documentos JSON?

- Importamos los datos en Mongo con el comando siguiente:

<code>mongoimport --db nb7-peliculas --collection movieDetails --drop --file <path/to/movieDetails.json></code>

A continuación, queremos realizar algunas consultas típicas para observar la diferencia entre el lenguaje de consulta SQL y el lenguaje de Mongo.

<b>Nota bena</b>: ¿Qué pasa si agregan la función <code>.pretty()</code> al final de un comando? Por ejemplo: <code>db.actors.find().pretty()</code>

1) Utilizar un comando Mongo para contar cuántos documentos hay en la colección 'movieDetails'. ¿Cuál sería el equivalente en SQL?

db.movieDetails.count()

SELECT count(*) as N from movieDetails;

2) Mostrar los títulos (title) y años de producción (year) de todas las peliculas de la colección. ¿A qué clausula SQL correspondría?

db.movieDetails.find({}, {_id: 0, title: 1, year: 1})

3) Contar las peliculas realizadas entre 1990 y 2000

db.movieDetails.count({$and: [{year: { $gte: 1990 }}, {year: { $lte: 2000 } } ] })


4) ¿Cuáles son los titulos de peliculas que ganaron más de 5 awards?

db.movieDetails.find({ "awards.wins": { $gt: 5 } }, { title: 1, "awards.wins": 1 })

5) ¿Cuál es el rating promedio en el sitio imdb para todas las peliculas?

db.movieDetails.aggregate({ $group: { _id: null, promedio: { $avg: "$imdb.rating" } } })

6) ¿Cuáles son las peliculas en las cuales participó el actor Nicolas Cage?

db.movieDetails.find({ actors: "Nicolas Cage" }, { title: 1 })

7) ¿Cuál es el número de pelicula por año?

db.movieDetails.aggregate([{$group: { _id: "$year", nPeliculas: { $sum: 1 } }}, { $sort: { _id: 1 } }])

8) ¿Cuál es la pelicula con el mejor rating?

db.movieDetails.find({}, { title: 1, _id: 0 }).sort({ "imdb.rating": -1 }).limit(1)

9) ¿Cuáles son los thrillers con un rating de al menos 6?

db.movieDetails.find({ $and: [{"imdb.rating": { $gte: 6 }}, { genres: "Thriller" }] }, {title: 1, _id: 0, "imdb.rating": 1 }).sort({ "imdb.rating": -1 })


10) ¿Cuáles son todos los valores de generos posibles?

db.movieDetails.distinct("genres")

11) ¿Cuáles son las peliculas que contienen la palabra 'Star'?

db.movieDetails.find({ title: { $in: [/star/i] } })

12) ¿Cuáles son los títulos que empiezan con la letra 'T'?

db.movieDetails.find({ title: { $in: [/^t/i] } }, { title: 1, _id: 0 })

13) ¿Cuáles son las peliculas que no son de tipo Drama o Thriller?

db.movieDetails.find({ $and: [{ genres: { $ne: "Drama" } }, { genres: { $ne: "Thriller" } }] }, { title: 1, _id: 0, genres: 1 })


## Ejercicio 3: Integrar Mongo con Python

Desde un entorno Python, conectarse a su base de datos movieDetails y realizar consultas similares al ejercicio 2.

In [1]:
from mongoengine import * 

connect('nb7-peliculas')


class tomato(EmbeddedDocument):
    meter = IntField()
    image = StringField()
    rating = IntField() 
    reviews = IntField() 
    fresh = IntField()
    consensus = StringField()
    userMeter = IntField()
    userRating = DecimalField()
    userReviews = IntField()
class imdb(EmbeddedDocument):
    id = StringField()
    StringField()
    rating = DecimalField()
    votes = IntField()
class awards(EmbeddedDocument):
    wins = IntField()
    nominations = IntField()
    text = StringField()
    oscars = IntField()
class movie_details(Document):
    title = StringField()
    year = IntField()
    rated = StringField()
    runtime = IntField()
    countries = ListField(StringField())
    genres = ListField(StringField())
    director = StringField()
    writers = ListField(StringField())
    actors = ListField(StringField())
    plot = StringField()
    poster = StringField()
    imdb = EmbeddedDocumentField(imdb)
    tomato = EmbeddedDocumentField(tomato)
    metacritic = IntField()
    awards = EmbeddedDocumentField(awards)
    type = StringField()

    

In [47]:
#1) Utilizar un comando Mongo para contar cuántos documentos hay en la colección 'movieDetails'. ¿Cuál sería el equivalente en SQL?
n = len(movie_details.objects)
print("Hay " + str(n) + " documentos en la coleccion")
#db.movie_details.find().count()

#2) Mostrar los títulos (title) y años de producción (year) de todas las peliculas de la colección. 
#¿A qué clausula SQL correspondría?
print("\n\nTitulos y años de produccion de todas las peliculas de la coleccion: ")
for i in movie_details.objects.limit(10):
    print(i.title + " " + str(i.year), end="\t")
#3) Contar las peliculas realizadas entre 1990 y 2000
c = 0
for i in movie_details.objects:
    if (i.year >= 1990 and i.year <= 2000):
        c+=1
print("\n\nHay " + str(c) + " peliculas realizadas entre 1990 y 2000")
#4) ¿Cuáles son los titulos de peliculas que ganaron más de 5 awards?
for i in movie_details.objects:
    if (i.awards):
        if (i.awards.wins > 5):
            #print(i.title)
            pass
#5) ¿Cuál es el rating promedio en el sitio imdb para todas las peliculas?
suma = 0
cont = 0
for i in movie_details.objects:
    if (i.imdb.rating):
        cont += 1
        suma += i.imdb.rating
print("\n\nEl promedio de rating promedio es: " + str(suma / cont))

#6) ¿Cuáles son las peliculas en las cuales participó el actor Nicolas Cage?

print("\n\nLas peliculas en las cuales participo el actor Nicolas Cage son:")
for i in movie_details.objects:
    for j in i.actors:
        if j == "Nicolas Cage":
            print("--" + i.title)


#7) ¿Cuál es el número de pelicula por año?

cont = 0
print("\nEl numero de películas por año son: \n")
for i in movie_details.objects.aggregate(*[{"$group": { "_id": "$year", "n": { "$sum": 1 } }}, \
                                           { "$sort": { "_id": 1 } }]):
    print(str(int(i["_id"])) + " => " + str(i["n"]) + "\t", end = "")
    cont += 1
    if cont == 6:
        print("")
        cont = 0




#8) ¿Cuál es la pelicula con el mejor rating?


best_imdb = movie_details.objects.order_by("-imdb.rating").limit(1)[0]
best_tomato = movie_details.objects.order_by("-tomato.rating").limit(1)[0]
print("\n\nLa mejor imdb con sort: " + best_imdb.title + " " + str(best_imdb.imdb.rating))
print("La mejor tomato con sort: " + best_tomato.title + " " + str(best_tomato.tomato.rating))
best_imdb = 0
best_imdb_title = ""
best_tomato = 0
best_tomato_title = ""
for i in movie_details.objects:
    if (i.imdb.rating and i.imdb.rating > best_imdb):
        best_imdb_title = i.title
        best_imdb = i.imdb.rating
    if ((i.tomato) and (i.tomato.rating > best_tomato)):
        best_tomato_title = i.title
        best_tomato = i.tomato.rating
print("\n\nLa mejor segun imdb es: " + best_imdb_title + " " + str(best_imdb))
print("La mejor segun tomato es: " + best_tomato_title + " " + str(best_tomato))
#9) ¿Cuáles son los thrillers con un rating de al meno 6?
lista = []
for i in movie_details.objects:
    for j in i.genres:
        if (j == "Thriller"):
            if ((i.imdb.rating and i.imdb.rating >= 6) or (i.tomato and i.tomato.rating >= 6)):
                lista.append(i.title)
            break
print("\n\nLas peliculas que son thrillers con un rating de al menos 6 son: ")
print(lista)

generos = {}
#10) ¿Cuáles son todos los valores de generos posibles?

print("\n\nTodos los valores de generos posibles son: ")

for i in movie_details.objects.distinct("genres"):
    print(i, end="\t")

#11) ¿Cuáles son las peliculas que contienen la palabra 'Star'?
titulos_star = []
for i in movie_details.objects:
    if ("star" in i.title.lower()):
        titulos_star.append(i.title)
print("\n\nLas peliculas que contienen la palabra star en su titulo son: ")
print(titulos_star)
#12) ¿Cuáles son los títulos que empiezan con la letra 'T'?

titulos_t = []
for i in movie_details.objects:
    if (i.title[0] == "T"):
        titulos_t.append(i.title)
print("\n\nLos titulos que empiezan con la letra T son: ")
print(titulos_t)
#13) ¿Cuáles son las peliculas que no son de tipo Drama o Thriller?
'''
for i in list(movie_details.objects(__raw__={"$and": [ { "genres": { "$ne": "Drama" } },{ "genres": { "$ne": "Thriller" } } ]})):
    print(i["title"], end="\t")
'''
  
titulos_no_drama_thriller = []
for i in movie_details.objects.limit(100):
    for j in i.genres:
        if (j == "Thriller") or (j == "Drama"):
            continue
        titulos_no_drama_thriller.append(i.title)
print("\n\nLas peliculas que no son drama o thriller son: ")
print(titulos_no_drama_thriller)

Hay 2295 documentos en la coleccion


Titulos y años de produccion de todas las peliculas de la coleccion: 
Once Upon a Time in the West 1968	Wild Wild West 1999	West Side Story 1961	Journey to the West 2013	Star Wars: Episode IV - A New Hope 1977	West of Memphis 2012	A Million Ways to Die in the West 2014	Star Wars: Episode V - The Empire Strikes Back 1980	Star Wars: Episode VI - Return of the Jedi 1983	Star Wars: Episode I - The Phantom Menace 1999	

Hay 372 peliculas realizadas entre 1990 y 2000


El promedio de rating promedio es: 6.503156049094097019286966686


Las peliculas en las cuales participo el actor Nicolas Cage son:
--Red Rock West
--Wild at Heart

El numero de películas por año son: 

1896 => 2	1897 => 2	1898 => 9	1899 => 4	1900 => 1	1901 => 6	
1902 => 5	1903 => 2	1904 => 1	1905 => 2	1906 => 2	1908 => 1	
1909 => 2	1910 => 2	1911 => 1	1912 => 2	1914 => 3	1915 => 6	
1916 => 1	1917 => 1	1918 => 1	1919 => 2	1920 => 1	1921 => 1	
1924 => 1	1925 => 5	1926 => 1	1927 => 1	1928 =>

## Ejercicio 4: Conceptos avanzados de Base de Datos: MySQL y Mongo

- <b>Indíce: optimización de consultas</b>

Referencias:
- Indíces en Mongo: https://docs.mongodb.com/manual/indexes/
- Indíces en MySQL: https://dev.mysql.com/doc/refman/8.0/en/optimization-indexes.html
- Introducción a indíces en MySQL: https://www.adictosaltrabajo.com/2015/09/11/introduccion-a-indices-en-mysql/

1) ¿Qué es un Indíce y de qué sirve?

## Un índice es una "lista" de referencias a ubicaciones y éstos sirven para acelerar el acceso a un documento/fila en específico dado la columna/dato indexado

2) ¿Cuándo los SGBDs utilizan los Indíces?

### Los SGDB utilizan los índices principalmente a la hora de los procesos de búsqueda.

3) ¿Cuáles son los tipos de Indíces en MySQL y Mongo?

### MySQL: PRIMARY KEY, UNIQUE, INDEX, and FULLTEXT
### Mongo: Single Field, Compound, Multikey, Text, 2d, Hashed.

4) Si los indíces son utiles, ¿por qué no crear indíces por cada columna/atributo? ¿Cuáles son los límimtes?

### Lo ideal es no crear índices para cada columna/atributo ya que esto en la práctica significa mas datos que guardar, lo que implica mayor espacio ocupado. Los límites estarían dados para cada sistema.

5) Conectarse a la base de datos 'sakila' de MySQL y ejecutar la consulta siguiente para poder conservar información sobre el rendimiento de consultas:

<code>set profiling=1;</code>

Ejecutar 10 veces la consulta siguiente:
<code>select * from rental where return_date>="2005-08-25 17:48:44";</code>

Luego, ejecutar el comando:
<code>show profiles;</code>

¿Cuál es la duración promedia del SGBD para responder a esta consulta?


### mean {0.00892821, 0.00910054, 0.01692007, 0.00881522, 0.01629027, 0.01601577, 0.01576211, 0.00872626, 0.00854397, 0.00885560} = 0.0117958

6) Crear un index sobre la columna 'return_date', y medir nuevamente 10 veces el rendimiento de la misma consulta. ¿Cuál es el nuevo valor promedio?

### mean {0.00729775, 0.00693777, 0.00784207, 0.00898454, 0.00912944, 0.00609408, 0.00913539, 0.00667461, 0.00894499, 0.00894222} = 0.00799829

7) ¿Cómo crear índices en Mongo?

### Single Field: db.records.createIndex( { location: 1 } )
### Compound Multikey: db.products.createIndex( { "item": 1, "stock": 1 } )
### Text: db.reviews.createIndex( { comments: "text" } )
### 2d: db.collection.createIndex( { location: "2d" } ,  { min : 0 , max : 15 } )
### Hashed: db.collection.createIndex( { _id: "hashed" } )  
  
 <br /> <br /> <br /> <br />
- <b>Transacciones</b>

Referencias:
- Introducción a Transacciones en MySQL: http://raknarrok.blogspot.com/2011/03/introduccion-transacciones.html
- Transacción en MySQL: https://dev.mysql.com/doc/refman/8.0/en/commit.html
- Transacción en Mongo: https://docs.mongodb.com/manual/core/transactions/

1) ¿Qué es una transacción? 

### Una transacción es la union de mas de una modificación en la base de datos, en la que si una de ellas falla, todas fallan, y si todas pasan, la transacción es exitosa.


2) ¿En términos de transacción, cuáles son las características que deberian garantizar un buen SGBD?

### La garantía de que las transacciones se hagan en el orden correcto, y un buen manejo de errores que permita el hecho de que si una falla, automaticamente se haga un rollback al estado anterior de la transacción.

3) En MySQL, crear una tabla <b>bank_user(name VARCHAR(10), account INT)</b> que permite conocer el monto disponible en la cuenta bancaria de algunos usuarios.
### create database bank;
### use bank;
### create table bank_user(name varchar(10), account int);
### insert into bank_user values("a", 250000);
### insert into bank_user values("b", 30000);

4) Realizar una operación que permita transferir 100.000 pesos del usuario A al usuario B. ¿Cómo garantizar que la operación no va a generar algún incoherencia en los datos? Escribir el código SQL correspondiente.

### start transaction;
### update bank_user set account=(account - 100000) where name="a";
### update bank_user set account=(account + 100000) where name="b";
### commit;

5) Supongamos que queremos hacerlo mismo en Mongo. ¿Mongo soporta transacciones ACID? ¿Desde cuándo? ¿Cómo se puede hacer transacciones? ¿Con qué limitaciones?

### Mongo soporta transacciones desde la version 4.0
### Se empieza una transacción usando Session.startTransaction()
### Se hace commit de una transacción con Session.commitTransaction()
### Se cancela manualmente una transacción con Session.abortTransaction()
### Las limitaciones que tienen las transacciones en Mongo son que no es posible ejecutar las operaciones que influyan el catalogo de bases de datos/colecciones, como crear una colección, borrar una colección y borrar indices.

<br/><br/><br/>
- <b>Sharding y Replicación</b>

1) ¿Cuál es el principio del Sharding?

### El principio de Sharding viene de la necesidad de particionar bases de datos muy grandes en pequeños grupos mas fáciles de manejar y guardar.

2) ¿Cuál es el principio de la Replicación?

### El principio de Replicación proviene de la necesidad del respaldo de la información y la seguridad de la existencia de ésta, o también aplica para tener un acceso mas rápido de alguna información sin necesidad de buscar lo mismo en otra ubicación.

3) ¿MySQL implementa mecanismos de Sharding/Replicación? y Mongo?

### MySQL en su forma clásica no implementa Sharding, pero si existen variaciones de ésta que permiten el escalar horizontalmente la base de datos, como lo es MySQL Cluster.
### MySQL si implementa replicación con la forma "Master-Slave" en la cual simplemente con el hecho que la base de datos maestra tenga acceso a la esclava, se encargará se hacer replicaciones de la información.
### Mongo se creó con el "Sharding" y "Replication" en mente por la necesidad de escalar horizontalmente aplicaciones que guarden y manejen mucha información, 